# Pose Estimation Models

## Imports

In [1]:
import pandas as pd
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_absolute_error, mean_squared_error

np.set_printoptions(suppress = True)

## Variables

In [2]:
data_path_prefix = "D:/Uni Stuff/IP/Data/"
image_path_prefix = data_path_prefix + "Blimp Images/Raw/image_"

test_split = 0.25

desired_image_size = 16

image_columns = []
colours = {0:"R", 1:"G", 2:"B"}

for i in range(desired_image_size):
    for j in range(desired_image_size):
        for k in range(3):
            image_columns.append(str(i + 1) + " " + str(j + 1) + " " + colours[k])

---

## Data Pre-processing

### CSVs

Load the bounding box and pose csv files and add to dataframes

In [4]:
bbox_data = pd.read_csv(data_path_prefix + "bbox data.csv")

pose_data = pd.read_csv(data_path_prefix + "blimp poses.csv")
y_variables = list(pose_data.columns)

### Images

Load the images from file, crop these by the bounding box and then resize to the desired square image size with padding to preserve aspect ratio. Image data is flattened and added to dataframe.

In [5]:
def cutout_image(image, bbox):
    width = image.shape[1]
    height = image.shape[0]

    center_x = bbox['cent_x'] * width
    center_y = bbox['cent_y'] * height

    bbox_width = bbox['width'] * width
    bbox_height = bbox['height'] * height

    min_x = np.clip(int(center_x - (bbox_width / 2)), 0, width)
    max_x = np.clip(int(center_x + (bbox_width / 2)), 0, width) 

    min_y = np.clip(int(center_y - (bbox_height / 2)), 0, height)
    max_y = np.clip(int(center_y + (bbox_height / 2)), 0, height)

    return image[min_y:max_y, min_x:max_x]

def process_image(image):
    old_size = image.shape[:2]

    size_ratio = float(desired_image_size) / max(old_size)

    new_size = tuple([int(x * size_ratio) for x in old_size])

    resized_image = cv.resize(image, (new_size[1], new_size[0]), interpolation=cv.INTER_AREA)

    delta_w = desired_image_size - new_size[1]
    delta_h = desired_image_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    padded_image = cv.copyMakeBorder(resized_image, top, bottom, left, right, cv.BORDER_CONSTANT, value=color)

    return padded_image

images = pd.DataFrame(columns=image_columns)

for i in range(len(pose_data)):
    img = cv.imread(image_path_prefix + str(i) + ".png")
    cropped = cutout_image(img, bbox_data.iloc[i])
    padded = process_image(cropped)

    images.loc[len(images)] = padded.flatten()

In [7]:
for i in range(5):
    img = cv.imread(image_path_prefix + str(i) + ".png")
    cropped = cutout_image(img, bbox_data.iloc[i])
    padded = process_image(cropped)

    cv.imwrite("D:/Uni Stuff/IP/image_" + str(i) + ".png", padded)

### Splits

Concatenate the bounding box data and image data into one dataframe. Split that and pose data into the training and testing lists.

In [7]:
X = pd.concat([bbox_data, images], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, pose_data, test_size=test_split)

## Cleanup

---

## Training

### Multilinear Regression

In [8]:
multilinear_model = linear_model.LinearRegression()
multilinear_model.fit(X_train, y_train)

LinearRegression()

### Ridge Regression

In [9]:
ridge_model = linear_model.Ridge()
ridge_model.fit(X_train, y_train)

Ridge()

### Lasso Regression

In [10]:
lasso_model = linear_model.Lasso()
lasso_model.fit(X_train, y_train)

c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.335e+04, tolerance: 3.095e+02
  model = cd_fast.enet_coordinate_descent(
c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.345e+03, tolerance: 2.266e+02
  model = cd_fast.enet_coordinate_descent(


Lasso()

### Polynomial Regression

In [10]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X_train.values)

poly_model = linear_model.LinearRegression()
poly_model.fit(poly_features, y_train)

In [ ]:
print(poly_features)

### Bayesian Regression

In [29]:
bayesian_model = []
bayesian_model = [linear_model.BayesianRidge() for i in range(len(y_variables))]

for i in range(len(y_variables)):
    bayesian_model[i].fit(X_train, y_train[y_variables[i]])

### Gaussian Regression

In [49]:
kernel = RBF()

gaussian_model = []
gaussian_model = [GaussianProcessRegressor(kernel=kernel) for i in range(len(y_variables))]

for i in range(len(y_variables)):
    bayesian_model[i].fit(X_train, y_train[y_variables[i]])

### Support Vector Regression

---

## Evaluating

### Multilinear Regression

Predict outputs of the testing split and determine mean squared and mean absolute errors.

In [15]:
multilinear_predictions = multilinear_model.predict(X_test)

print('mean_squared_error : ', mean_squared_error(y_test, multilinear_predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, multilinear_predictions))

mean_squared_error :  43.385313427133305
mean_absolute_error :  3.5946233031205463


In [ ]:
i = 2341

print("Actual:    " + str(y_test.iloc[i].values))
print("Predicted: " + str(multilinear_predictions[i]))

### Ridge Regression

In [16]:
ridge_predictions = ridge_model.predict(X_test)

print('mean_squared_error : ', mean_squared_error(y_test, ridge_predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, ridge_predictions))

mean_squared_error :  43.25735661472114
mean_absolute_error :  3.5974812170953823


### Lasso Regression

In [17]:
lasso_predictions = lasso_model.predict(X_test)

print('mean_squared_error : ', mean_squared_error(y_test, lasso_predictions))
print('mean_absolute_error : ', mean_absolute_error(y_test, lasso_predictions))

mean_squared_error :  63.16263601347481
mean_absolute_error :  4.36555379872854


### Polynomial Regression

### Bayesian Regression

In [36]:
bayesian_predictions = []
bayesian_predictions = [bayesian_model[i].predict(X_test) for i in range(len(bayesian_model))]

mean_squared_errors = [0] * len(bayesian_predictions)
mean_absolute_errors = [0] * len(bayesian_predictions)

for i in range(len(bayesian_predictions)):
    mean_squared_errors[i] = mean_squared_error(y_test[y_variables[i]], bayesian_predictions[i])
    mean_absolute_errors[i] = mean_absolute_error(y_test[y_variables[i]], bayesian_predictions[i])

print('mean_squared_errors : ', mean_squared_errors)
print('mean_absolute_errors : ', mean_absolute_errors)

print('worst_mean_squared_error : ', max(mean_squared_errors))
print('worst_mean_absolute_error : ', max(mean_absolute_errors))

mean_squared_errors :  [174.70540447615963, 64.32983196345607, 22.608114043835577, 0.007483105411069927, 0.015270915414334826, 1.1324843130903621]
mean_absolute_errors :  [11.091434685967984, 6.2501243924163585, 3.656565108966513, 0.07088285144724372, 0.09333767147060837, 0.6046466527538764]
worst_mean_squared_error :  174.70540447615963
worst_mean_absolute_error :  11.091434685967984


### Gaussian Regression

In [50]:
gaussian_predictions = []
gaussian_predictions = [gaussian_model[i].predict(X_test) for i in range(len(gaussian_model))]

mean_squared_errors = [0] * len(gaussian_predictions)
mean_absolute_errors = [0] * len(gaussian_predictions)

for i in range(len(gaussian_predictions)):
    mean_squared_errors[i] = mean_squared_error(y_test[y_variables[i]], gaussian_predictions[i])
    mean_absolute_errors[i] = mean_absolute_error(y_test[y_variables[i]], gaussian_predictions[i])

print('mean_squared_errors : ', mean_squared_errors)
print('mean_absolute_errors : ', mean_absolute_errors)

print('worst_mean_squared_error : ', max(mean_squared_errors))
print('worst_mean_absolute_error : ', max(mean_absolute_errors))

mean_squared_errors :  [2037.825542655617, 298.36466432742714, 96.98200478024835, 0.010264234077006142, 0.04115380083531885, 3.326177908224425]
mean_absolute_errors :  [40.357642735481264, 13.363598386759124, 7.677266850466887, 0.08751113504250783, 0.17583206218304695, 1.5824390139432392]
worst_mean_squared_error :  2037.825542655617
worst_mean_absolute_error :  40.357642735481264


c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(
c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(
c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(
c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without feature names
  warnings.warn(
c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GaussianProcessRegressor was fitted without f

---

## Tuning

### Ridge Regression

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)

search = GridSearchCV(ridge_model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

results = search.fit(X_train, y_train)

print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)